In [1]:
import tensorflow as tf

In [2]:
# Huber loss: Robust to outliers
def huber_loss(labels,predictions,delta=1.0):
    residual = tf.abs(prediction - labels)
    condition = tf.less(residual,delta)
    small_res = 0.5 * tf.square(residual)
    large_res = delta * residual - 0.5 * tf.square(delta)
    return tf.select(condition, samll_res, large_res)

In [2]:
# logistic regression
# MNIST Database
# logits = X * w + b, Y_predicted = softmax(logits), loss = cross_entropy(Y, Y_predicted)
# Batch them up!!!
learning_rate = 0.01
batch_size = 128
n_epochs = 30

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32,[batch_size,784],name="image")
Y = tf.placeholder(tf.float32,[batch_size,10],name="label")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
import time
w = tf.Variable(tf.random_normal(shape=[784,10],stddev=0.01),name = 'weights')
b = tf.Variable(tf.zeros([1,10]),name="bias")
logits = tf.matmul(X,w) + b
# define loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y,name='loss')
loss = tf.reduce_mean(entropy) #computes the mean over all the examples in the batch

#define training op
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    # to visualize
    #writer = tf.summary.FileWriter('logistic_reg',sess.graph)
    start_time=time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs):
        total_loss = 0
        for _ in range(n_batches):
            X_batch,Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer,loss],feed_dict = {X:X_batch, Y:Y_batch})
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs
    
    # test the model

    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) 

    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0

    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch

    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

    #writer.close()

Average loss epoch 0: 1.288614780058116
Average loss epoch 1: 0.7321287525561584
Average loss epoch 2: 0.5990042393302029
Average loss epoch 3: 0.5358560870299528
Average loss epoch 4: 0.49729369757892367
Average loss epoch 5: 0.4703630122132513
Average loss epoch 6: 0.45180987401719974
Average loss epoch 7: 0.4344715336402813
Average loss epoch 8: 0.4230245681075783
Average loss epoch 9: 0.4147163439880718
Average loss epoch 10: 0.40201089105683885
Average loss epoch 11: 0.39903330747342053
Average loss epoch 12: 0.39047330336554065
Average loss epoch 13: 0.38301682312449653
Average loss epoch 14: 0.3783457083649291
Average loss epoch 15: 0.37345087490020656
Average loss epoch 16: 0.3694598871719587
Average loss epoch 17: 0.36933862707970583
Average loss epoch 18: 0.36019824785647137
Average loss epoch 19: 0.3616117585650135
Average loss epoch 20: 0.3555428270415548
Average loss epoch 21: 0.3522934715300451
Average loss epoch 22: 0.35303924255298846
Average loss epoch 23: 0.3477310033

In [120]:
import numpy as np
pre = np.array([[31, 23,  4, 24, 27, 34],
                [18,  3, 25,  0,  6, 35],
                [28, 14, 33, 22, 20,  8],
                [13, 30, 21, 19,  7,  9],
                [16,  1, 26, 32,  2, 29],
                [17, 12,  5, 11, 10, 15]])

pre1 = np.array([[31, 23,  4, 24, 27, 34],
                [16,  1, 26, 32,  2, 29]])

pre2 = np.array([[17, 12,  5, 11, 10, 15]])

pre3 = np.array([[0.1,0.7,30,13]])

y = np.array([[31, 23,  4, 24, 27, 34],
                [18,  3, 25,  0,  6, 35],
                [28, 14, 33, 22, 20,  8],
                [13, 30, 21, 19,  7,  9],
                [16,  1, 26, 32,  2, 29],
                [1, 2,  5, 11, 10, 15]])
a = tf.argmax(pre,1)
b = tf.argmax(pre1,0) # transpose the data
c = tf.argmax(pre2,1) # see the data through row.
d = tf.argmax(pre3,1)
e = tf.argmax(y,1)
f = tf.argmax(pre,0)

In [122]:
# tf.argmax(,1) returns the location of the largest value,[31, 23,  4, 24, 27, 34]
# 34 is the 5th number in python.
# tf.argmax(,0) transpose the input data
tf.InteractiveSession()
b.eval()

array([0, 0, 1, 1, 0, 0], dtype=int64)

In [93]:
aaa= tf.equal(a,e).eval()
tf.cast(aaa, tf.float32).eval()
tf.reduce_sum(tf.cast(aaa, tf.float32)).eval()

5.0